In [3]:
import numpy as np
import tensorflow as tf 
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from sklearn.datasets import make_blobs

2024-06-08 12:29:18.651924: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
# make 4-class dataset for classification
classes = 4
m = 100
centers = [[-5, 2], [-2, -2], [1, 2], [5, -2]]
std = 1.0
X_train, y_train = make_blobs(n_samples=m, centers=centers, cluster_std=std,random_state=30)

In [5]:
# show classes in data set
print(f"unique classes {np.unique(y_train)}")
# show how classes are represented
print(f"class representation {y_train[:10]}")
# show shapes of our dataset
print(f"shape of X_train: {X_train.shape}, shape of y_train: {y_train.shape}")

unique classes [0 1 2 3]
class representation [3 3 3 0 3 3 3 3 2 0]
shape of X_train: (100, 2), shape of y_train: (100,)


In [6]:
model = Sequential([
    Dense(units = 25, activation = 'relu'),
    Dense(units = 15, activation = 'relu'),
    Dense(units = 4, activation = 'linear')
])

model.compile(
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = tf.keras.optimizers.Adam(0.001)
)

model.fit(
    X_train,y_train,
    epochs = 10
)

model.summary()

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 1.4416
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3258 
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1797 
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1031 
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9819 
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9701 
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.8654 
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7934 
Epoch 9/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.7555 
Epoch 10/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.7298 


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 25)             │            75 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           390 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │            64 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,589 (6.21 KB)

 Trainable params: 529 (2.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,060 (4.14 KB)

In [7]:
# The output it currently linear, not a probability
linear_output = model.predict(X_train)
print(f"Two Examples of Linear Output:\n {linear_output[:2]}")

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
Two Examples of Linear Output:
 [[-4.2251563 -1.2639108 -1.4196199  0.7621225]
 [-3.9782484 -1.1989932 -1.6148388  0.6444655]]


In [8]:
# Therefore, we must send our linear output through softmax to convert to probabilities
softmax_output = tf.nn.softmax(linear_output)
print(f"Two Examples of Softmax Output:\n {softmax_output[:2]}")

Two Examples of Softmax Output:
 [[0.00545271 0.10535736 0.09016568 0.7990243 ]
 [0.00772177 0.1243747  0.0820602  0.7858433 ]]


In [9]:
# Another way to format the last two blocks of code is this
output = tf.nn.softmax(model(X_train))
print(f"Two Examples of Output:\n {output[:2]}")

Two Examples of Output:
 [[0.00545271 0.10535736 0.09016568 0.7990243 ]
 [0.00772177 0.1243747  0.0820602  0.7858433 ]]


In [10]:
# Converting to Softmax is not necessary for selecting the most likely catagory
# Instead, we can use np.argmax(), which returns the indices of the max values
# The same catagories would be outputted it we used out softmax output
for i in range(5):
    print(f"{linear_output[i]}, catagory: {np.argmax(linear_output[i])}")


[-4.2251563 -1.2639108 -1.4196199  0.7621225], catagory: 3
[-3.9782484 -1.1989932 -1.6148388  0.6444655], catagory: 3
[-4.387918   -1.3148063  -1.547532    0.77385646], catagory: 3
[ 1.1219547  -0.5501515   0.00118462 -0.9366633 ], catagory: 0
[-4.0460763  -0.84620243  0.21037278  1.2464452 ], catagory: 3


In [11]:
# Testing
X_test = np.array([
    [1,2],
    [5,0.5]
])
prediction = tf.nn.softmax(model(X_test))
for i in range(X_test.shape[0]):
    print(f"Prediction for Test {i}:\n {prediction[i]}, Catagory: {np.argmax(prediction[i])}")


Prediction for Test 0:
 [0.10617839 0.12200226 0.52379125 0.24802803], Catagory: 2
Prediction for Test 1:
 [0.00123485 0.05507851 0.29146788 0.65221876], Catagory: 3
